In [1]:
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

import re

from datetime import datetime

SEASONS = list(range(2008,2024))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [16]:
driver = webdriver.Firefox()

for season in SEASONS[-1:]:

    driver.get(f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html")

    WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
    )

    parsed = BeautifulSoup(driver.page_source)

    standings = [x["href"] for x in parsed.find("div", attrs={"class": "filter"}).find_all("a")]

    for standing in standings:
        url = f"https://www.basketball-reference.com" + standing

        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        driver.get(url)

        WebDriverWait(driver, 10).until(
                lambda driver: driver.execute_script("return document.readyState") == "complete"
        )

        parsed = BeautifulSoup(driver.page_source)

        schedule = parsed.find("div", attrs={"id":"all_schedule"})

        with open(save_path, "w+") as f:
             f.write(str(schedule))

In [2]:
standings_files = os.listdir(STANDINGS_DIR)

In [3]:
driver = webdriver.Firefox()

In [9]:
for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        with open(os.path.join(STANDINGS_DIR, f), 'r') as f:
            html = f.read()

        soup = BeautifulSoup(html)
        links = soup.find_all("a")
        hrefs = [l.get('href') for l in links]
        
        box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

        for url in box_scores:
            save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
            if os.path.exists(save_path):
                continue

            driver.get(url)

            WebDriverWait(driver, 10).until(
                    lambda driver: driver.execute_script("return document.readyState") == "complete"
            )

            parsed = BeautifulSoup(driver.page_source)

            with open(save_path, "w+") as f:
                f.write(str(parsed))

            time.sleep(3)